# Within Salish Sea SSC and CIOPS compare

In [1]:
import numpy as np
import xarray as xr
import scipy.interpolate as interp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_data(mydata, depthf, section, starttime, endtime):
    depths = depthf(mydata.final_z[(mydata.final_section==section) & (mydata.init_t >= starttime) & (mydata.init_t <= endtime)]-1.)[0]
    transports = mydata.final_transp[(mydata.final_section==section) & (mydata.init_t >= starttime) & (mydata.init_t <= endtime)]
    temp = mydata.final_temp[(mydata.final_section==section) & (mydata.init_t >= starttime) & (mydata.init_t <= endtime)]
    sal = mydata.final_salt[(mydata.final_section==section) & (mydata.init_t >= starttime) & (mydata.init_t <= endtime)]
    return depths, transports, temp, sal

In [7]:
def get_all_data(file, mymesh):
    
    depthf = interp.interp1d(mymesh.z, mymesh.gdept_1d)
    
    yearjumps = [0,1,-2,1,0,1,0,1,1,0,1,0]
    starttime = 1
    endtime = 720
    sections = [0,2,3,4,5,6] #meander, admiralty, deception, rosario, sanjuan, and haro (same for both models)

    depths = np.array([])
    transports = np.array([])
    temp = np.array([])
    sal = np.array([])
    month = np.array([])
    sec = np.array([])

    mydata = xr.open_dataset(file)
    for i in range(12):
        for section in sections:
            tempD, tempT, tempt1, tempt2 = get_data(mydata, depthf, section, starttime+(i*720+(sum(yearjumps[:i+1]*24))), endtime+(i*720+(sum(yearjumps[:i+1]*24))))

            depths = np.append(depths, tempD)
            transports = np.append(transports, tempT/720)
            temp = np.append(temp, tempt1)
            sal = np.append(sal, tempt2)
            month = np.append(month, np.full(len(tempD),i+1))
            sec = np.append(sec, np.full(len(tempD),section))
    
    return depths, transports, month, temp, sal, sec

In [4]:
mymesh_SSC = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')

mymesh_CIOPS = xr.open_dataset('/ocean/mdunphy/CIOPSW-BC12/grid/mesh_mask_Bathymetry_NEP36_714x1020_SRTM30v11_NOAA3sec_WCTSS_JdeFSalSea.nc')

In [5]:
file_SSC = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/1yr_runs/201905_1hr/forward_01jan17/ariane_positions_quantitative.nc'
file_CIOPS = '/ocean/rbeutel/MOAD/analysis-becca/Ariane/CIOPS_SS_1yr/forward_01jan17/ariane_positions_quantitative.nc'

In [9]:
depths, transports, month, temp, sal, sec = get_all_data(file_SSC, mymesh_SSC)
d = {'month':month, 'section':sec, 'transport': transports, 'temperature':temp, 'salinity':sal, 'depth':depths}
SSC = pd.DataFrame(data=d)

In [ ]:
depths, transports, month, temp, sal, sec = get_all_data(file_CIOPS, mymesh_CIOPS)
d = {'month':month, 'section':sec, 'transport': transports, 'temperature':temp, 'salinity':sal, 'depth':depths}
CIOPS = pd.DataFrame(data=d)